# Extension Exercises


<div align="right"><button><a href="https://colab.research.google.com/github/QuantEcon/workshop.africa-july2023/blob/main/day-10/exercise_set_10_with_solution.ipynb"><img src="" heght="10px"/><img
  src="https://colab.research.google.com/assets/colab-badge.svg"
  alt="open with Colab" width="100px"/></a></button></div>

#### Written for the QuantEcon Africa Workshop (July 2023)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon

## Exercise 1

***Exercise 1.1*** Consider the following setup:

Given we have no prior knowledge about whether a coin is fair or unfair (that is we do not know whether the coin has a probability of head (p) equals to 0.5 or other values). In light of this prior belief, we assign a uniform prior to the probability of heads, $\theta$. 

Your task is to write a function that takes a sequence of observations of coin flips, update belief after each flip, and returns the posterior distribution of $\theta$ on a grid of points between 0 and 1. Set the true value of $\theta$ to 0.7 and the number of coin flips ($n$) to 10000.

(Hint: think about what distribution does coin flip follow?)

### Solution

In [ ]:
n = 50_000 # Number of coin flips
bias = 0.7 # The true bias

xs = np.linspace(0, 1, 101) # The grid
θ = np.repeat(1/len(xs), len(xs)) # Uniform θ

def coin_flips(n, bias):
    # Simulate n coin flips with bias
    return np.random.choice([0, 1], size=n, p=[1-bias, bias])

flips = coin_flips(n, bias)

posterior_list = []
for flip in flips:
    likelihood = xs**flip * (1-xs)**(1-flip) # Bernoulli Likelihood
    updated_beliefs = likelihood * θ
    posterior = likelihood * θ / np.sum(updated_beliefs)
    posterior_list.append(θ)
    θ = posterior
posterior = posterior_list[-1]

plt.plot(xs, posterior, color='black', alpha=0.4)
plt.vlines(bias, 0, np.max(posterior), color='black', linestyles='--')
plt.show()

***Exercise 1.2*** Now instead of assuming a uniform prior on $[0, 1]$, assume the grid is restricted  to $[0.6, 0.8]$. What is the posterior distribution of $\theta$ after 100, 200, ..., 10000 coin flips? Visualize the distribution if you can.

### Solution

In [ ]:
n = 10_000 # Number of coin flips
bias = 0.7 # The true bias

def simulate(n, bias):
    xs = np.linspace(0.6, 0.8, 101) # The grid
    θ = np.repeat(1/len(xs), len(xs)) # Uniform θ

    def coin_flips(n, bias):
        # Simulate n coin flips with bias
        return np.random.choice([0, 1], size=n, p=[1-bias, bias])

    flips = coin_flips(n, bias)

    posterior_list = []
    for flip in flips:
        likelihood = xs**flip * (1-xs)**(1-flip) # Bernoulli Likelihood
        updated_beliefs = likelihood * θ
        θ = likelihood * θ / np.sum(updated_beliefs)
        posterior_list.append(θ)

    posterior = posterior_list[-1]
    plt.plot(xs, posterior, color='grey', alpha=0.2)

for i in range(100, n+1, 100):
    if i % 2000 == 0:
        print(f"Simulating {i} coin flips...")
    simulate(i, bias)

plt.axvline(bias, color='black')
plt.show()


***Exercise 1.3*** [Beta-binomial model](https://en.wikipedia.org/wiki/Beta-binomial_distribution#Beta-binomial_in_Bayesian_statistics) is well studied due to their conjugacy. Please show that the update in Exercise 1.1 fit into the construct of beta-binomial model.

### Solution

$\theta |_{\alpha = \beta = 1} \sim Unif(0, 1)$ in Exercise 1.1.

## Exercise 2

In this exercise, please show the fact we noticed in Exercise 1.3 mathematically.

***Exercise 2.1*** Please write down the likelihood function for $\theta$ after observing one flip of our coin.

### Solution

Suppose the outcome is Y, then the likelihood function is

$$
L(Y|\theta)= \textrm{Prob}(X =  Y | \theta) = 
\theta^Y (1-\theta)^{1-Y}
$$

***Exercise 2.2*** Please write the prior distribution (in terms of $\alpha$ and $\beta$) for $\theta$.

### Solution

The prior distribution is

$$
\textrm{Prob}(\theta) = \frac{\theta^{\alpha - 1} (1 - \theta)^{\beta - 1}}{B(\alpha, \beta)}
$$

***Exercise 2.3*** Please write the posterior distribution for $\theta$ after observing one flip of our coin.

### Solution

We can derive the posterior distribution for $\theta$ via 

\begin{align*}
  \textrm{Prob}(\theta | Y) &= \frac{\textrm{Prob}(Y | \theta) \textrm{Prob}(\theta)}{\textrm{Prob}(Y)} \\
  &=\frac{\textrm{Prob}(Y | \theta) \textrm{Prob}(\theta)}{\int_{0}^{1} \textrm{Prob}(Y | \theta) \textrm{Prob}(\theta) d \theta }\\
  &= \frac{\theta^Y (1-\theta)^{1-Y}\frac{\theta^{\alpha - 1} (1 - \theta)^{\beta - 1}}{B(\alpha, \beta)}}{\int_{0}^{1}\theta^Y (1-\theta)^{1-Y}\frac{\theta^{\alpha - 1} (1 - \theta)^{\beta - 1}}{B(\alpha, \beta)} d \theta } \\
  &= \frac{ \theta^{Y+\alpha - 1} (1 - \theta)^{1-Y+\beta - 1}}{\int_{0}^{1}\theta^{Y+\alpha - 1} (1 - \theta)^{1-Y+\beta - 1} d \theta}
\end{align*}

which means that

$$
\textrm{Prob}(\theta | Y) \sim \textrm{Beta}(\alpha + Y, \beta + (1-Y))
$$


## Exercise 3

[Inverse transform sampling](https://python.quantecon.org/prob_matrix.html#classic-trick-for-generating-random-numbers) is a classic method used to generate random numbers from a desired probability distribution. The basic idea behind inverse transform sampling is to transform uniform random numbers into random numbers that follow the desired distribution.


The core idea is that if we draw a random number $u$ from a uniform distribution on $[0, 1]$, then the random variable $F^{-1}(u)$ has the distribution $F$:

$$
X=F^{-1}(U)
$$

where X is a random variable with CDF $F$:

$$
F_X(x)=F(x)=\textrm{Prob}\{X\le x\}
$$

A general algorithm for inverse transform sampling is as follows:

1. Draw a random number $u$ from a uniform distribution on $[0, 1]$.

2. Compute $x=F^{-1}(u)$.

3. $x$ is a random number from the distribution $F$.

Implement the inverse transform sampling algorithm for the exponential distribution with parameter $\lambda$ = 0.5

### Solution

In [ ]:
λ = 0.5

# Number of random samples to generate
num_samples = 1000

# Inverse transform sampling
def inverse_transform_sampling(num_samples, λ):
    u = np.random.uniform(0, 1, num_samples)  # Generate uniform random numbers
    x = -np.log(1 - u) / λ  # Apply inverse transform to get exponential random numbers
    return x

# Generate random samples
samples = inverse_transform_sampling(num_samples, λ)

# Plot the histogram of the generated samples
plt.hist(samples, bins=30, density=True, alpha=0.5, label='Generated samples')

x = np.linspace(0, 10, 1000)
plt.plot(x, expon.pdf(x, scale=1/λ),
       'r-', lw=2, alpha=0.5, label='expon pdf')

plt.xlabel('x')
plt.ylabel('Probability density')
plt.title('Inverse Transform Sampling')
plt.legend()
plt.show()


## Exercise 4

Consider the stochastic second-order linear difference equation

$$
y_{t} = \alpha_{0} + \alpha_{1} y_{y-1} + \alpha_{2} y_{t-2} + u_{t}
$$

where $u_{t} \sim N \left(0, \sigma_{u}^{2}\right)$ and

$$
\left[\begin{array}{c}
y_{-1}\\
y_{0}
\end{array}\right]\sim N\left(\mu_{\tilde{y}},\Sigma_{\tilde{y}}\right)
$$

It can be written as a stacked system

$$
\underset{\equiv A}{\underbrace{\left[\begin{array}{cccccccc}
1 & 0 & 0 & 0 & \cdots & 0 & 0 & 0\\
-\alpha_{1} & 1 & 0 & 0 & \cdots & 0 & 0 & 0\\
-\alpha_{2} & -\alpha_{1} & 1 & 0 & \cdots & 0 & 0 & 0\\
0 & -\alpha_{2} & -\alpha_{1} & 1 & \cdots & 0 & 0 & 0\\
\vdots & \vdots & \vdots & \vdots & \cdots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & 0 & \cdots & -\alpha_{2} & -\alpha_{1} & 1
\end{array}\right]}}\left[\begin{array}{c}
y_{1}\\
y_{2}\\
y_{3}\\
y_{4}\\
\vdots\\
y_{T}
\end{array}\right]=\underset{\equiv b}{\underbrace{\left[\begin{array}{c}
\alpha_{0}+\alpha_{1}y_{0}+\alpha_{2}y_{-1}\\
\alpha_{0}+\alpha_{2}y_{0}\\
\alpha_{0}\\
\alpha_{0}\\
\vdots\\
\alpha_{0}
\end{array}\right]}}
$$



***Exercise 4.1*** Code the stacked system using the parameters below as $A$ and calculate the inverse of the system.

### Solution

In [ ]:
# set parameters
T = 160

# coefficients of the second order difference equation
𝛼0 = 10
𝛼1 = 1.53
𝛼2 = -.9

# variance of u (\sigma_u^2)
σu = 10.

# distribution of y_{-1} and y_{0}
μy_tilde = np.array([1., 0.5])
Σy_tilde = np.array([[2., 1.], [1., 0.5]])

In [ ]:
# construct A and A^{\prime}
A = np.zeros((T, T))

for i in range(T):
    A[i, i] = 1

    if i-1 >= 0:
        A[i, i-1] = -𝛼1

    if i-2 >= 0:
        A[i, i-2] = -𝛼2

A_inv = np.linalg.inv(A)

***Exercise 4.2*** We can compute $y$ by solving the system

$$
y = A^{-1} \left(b + u\right)
$$

Please solve for $\mu_{y}$ and $\Sigma_{y}$.


### Solution

We have

$$
\begin{aligned}
\mu_{y} &= A^{-1} \mu_{b} \\
\Sigma_{y} &= A^{-1} E \left[\left(b - \mu_{b} + u \right) \left(b - \mu_{b} + u \right)^{\prime}\right] \left(A^{-1}\right)^{\prime} \\
           &= A^{-1} \left(\Sigma_{b} + \Sigma_{u} \right) \left(A^{-1}\right)^{\prime}
\end{aligned}
$$

where

$$
\mu_{b}=\left[\begin{array}{c}
\alpha_{0}+\alpha_{1}\mu_{y_{0}}+\alpha_{2}\mu_{y_{-1}}\\
\alpha_{0}+\alpha_{2}\mu_{y_{0}}\\
\alpha_{0}\\
\vdots\\
\alpha_{0}
\end{array}\right]
$$

$$
\Sigma_{b}=\left[\begin{array}{cc}
C\Sigma_{\tilde{y}}C^{\prime} & \boldsymbol{0}_{N-2\times N-2}\\
\boldsymbol{0}_{N-2\times2} & \boldsymbol{0}_{N-2\times N-2}
\end{array}\right],\quad C=\left[\begin{array}{cc}
\alpha_{2} & \alpha_{1}\\
0 & \alpha_{2}
\end{array}\right]
$$

$$
\Sigma_{u}=\left[\begin{array}{cccc}
\sigma_{u}^{2} & 0 & \cdots & 0\\
0 & \sigma_{u}^{2} & \cdots & 0\\
\vdots & \vdots & \vdots & \vdots\\
0 & 0 & \cdots & \sigma_{u}^{2}
\end{array}\right]
$$


In [ ]:
# compute the mean vectors of b and y
μb = np.full(T, 𝛼0)
μb[0] += 𝛼1 * μy_tilde[1] + 𝛼2 * μy_tilde[0]
μb[1] += 𝛼2 * μy_tilde[1]

μy = A_inv @ μb

In [ ]:
# compute the covariance matrices of b and y
Σu = np.eye(T) * σu ** 2

Σb = np.zeros((T, T))

C = np.array([[𝛼2, 𝛼1], [0, 𝛼2]])
Σb[:2, :2] = C @ Σy_tilde @ C.T

Σy = A_inv @ (Σb + Σu) @ A_inv.T

## Exercise 5

Congratulations for making it to the end of the workshop! We hope you have enjoyed the workshop and learned a lot.

Please run through [more advanced QuantEcon lecture series](https://quantecon.org/projects/#filter=lecture) and see what you can do with the tools you have learned in this workshop.


